# Efficiency - Add a single point

| comparable algo list |
| - |
| pivot |
| delta |
| Monte Carlo |

In [ ]:
import sys
sys.path.append("../..")

from sklearn import svm
import numpy as np
from copy import deepcopy

import src

from src.data_utils import (
    load_npy, save_npy, preprocess_data, variance, normalize, generate_union_description
)

In [ ]:
model = svm.SVC(decision_function_shape='ovo')

size_choices = [10, 100, 1000, 10000]
m_choices = [10, 15, 18, 20, 22, 25, 28, 30]

stone_m = 100

for size in size_choices:
    x_train, y_train, x_test, y_test, \
    columns_name = preprocess_data(
        't_train_' + str(size) + 'p.csv',
        't_test_' + str(size) + 'p.csv')

    # process data
    x_train_ = x_train[:-1]
    y_train_ = y_train[:-1]

    # for add
    add_point_x = x_train[-1]
    add_point_y = y_train[-1]

    x_test_ = x_test
    y_test_ = y_test

    union_d = generate_union_description(len(x_train), 10)
    union_d_ = deepcopy(union_d)
    for idx in [len(x_train)]:
        for _ in union_d_:
            if idx in _:
                _.remove(idx)
                break

    # get a init ov
    init_ov = src.mc_owen(x_train_, y_train_, x_test_, y_test_, model, union_d_, stone_m * len(y_train_), proc_num=2)
    save_npy('as_init_mc_plus_size' + str(size) + '.npy', init_ov)

    # set a stone
    mc_plus = src.mc_owen(x_train, y_train, x_test_, y_test_, model, union_d, stone_m * len(y_train), proc_num=2)
    save_npy('as_mc_plus_size' + str(size) + '.npy', mc_plus)

    # left ov for pivot
    pivot_owen = src.PivotOwen(x_train_, y_train_, x_test_, y_test_, model,
                                   None, union_d_)
    lov = pivot_owen.prepare(m=len(y_train_)*stone_m)
    save_npy('as_pivot_lov_size' + str(size) + '.npy', lov)

    # mc
    mc = src.mc_owen(x_train, y_train, x_test_, y_test_, model, union_d, stone_m * len(y_train))
    save_npy('as_mc'+ str(size) + 'm' + str(stone_m) + '.npy', mc)

    # # walk the m choices
    for m in m_choices:

        # delta
        delta_owen = src.DeltaOwen(x_train_, y_train_, x_test_, y_test_, model,
                                        init_ov, union_d_)
        delta_ov = delta_owen.add_single_point(add_point_x,
                                               add_point_y,
                                               4,
                                               m=(len(y_train_)+1)*m)
        save_npy('as_delta_size' + str(size) +'m' + str(m) + '.npy', delta_ov)

        # pivot
        pivot_owen = src.PivotOwen(x_train_, y_train_, x_test_, y_test_, model,
                                        None, union_d_)
        pivot_owen.lov = lov
        pivot_ov = pivot_owen.add_single_point(add_point_x, add_point_y, 4,
                                               m=(len(y_train_)+1)*m,
                                               flags={'flag_lov': True})
        save_npy('as_pivot_size' + str(size) +'m' + str(m) + '.npy', pivot_ov)